In [72]:
import numpy as np
import tensorflow as tf
from tensorflow.contrib import rnn
import random
import collections
import time

In [85]:
exp = 'esp' 

In [86]:
def read_data(fname):
    with open(fname) as f:
        content = f.readlines()
    content = [x.strip() for x in content]
    content = [word for i in range(len(content)) for word in content[i].split()]
    content = np.array(content)
    return content

def build_dataset(words):
    count = collections.Counter(words).most_common()
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return dictionary, reverse_dictionary

In [87]:
# Text file containing words for training
if exp == 'cat_2':
    training_file = 'belling_the_cat.txt'
else:
    training_file = 'train_text.txt'
training_data = read_data(training_file)
dictionary, reverse_dictionary = build_dataset(training_data)
print("Data loaded")

Data loaded


In [88]:
batch_size = 2000
base = './'+exp+'/checkpoints/' # directory of saved model
name = 'model_final' #last trained model

In [89]:
import pandas as pd
df = pd.DataFrame.from_dict(dictionary, orient='index')
df.columns = ['id']
df.head() #you can delete head() method to display all words

,id
chilaca,445
cardemil,1017
debut,447
vayan,449
relator,257


In [90]:
direct = base+name+'.meta'
saver = tf.train.import_meta_graph(direct)
with tf.Session() as sess:
    ckpts = tf.train.get_checkpoint_state(base)
    saver.restore(sess, save_path=base+name)

    all_vars = tf.get_collection('placeholders')
    inputs = all_vars[0]
    labels = all_vars[1]


    all_vars2 = tf.get_collection('loss_computation')
    cost = all_vars2[0]
    acc = all_vars2[1]
    pred = all_vars2[2]

    n_input = 3

    while True:
        prompt = "%s words: " % n_input
        sentence = str(input(prompt))
        print ('sentence: ', sentence)
        sentence = sentence.strip()
        words = sentence.split(' ')
        print (words)
        if len(words) != n_input:
            continue

        symbols_in_keys = [dictionary[str(words[i])] for i in range(len(words))]
        for i in range(32):
            keys = np.reshape(np.array(symbols_in_keys), [-1, n_input, 1])
            onehot_pred = sess.run([pred], feed_dict={inputs: keys})
            onehot = np.array(onehot_pred)
            onehot = np.reshape(onehot, (1,onehot.shape[2]))
            value  = tf.argmax(onehot, 1)

            onehot_pred_index = int(value.eval())
            sentence = "%s %s" % (sentence, reverse_dictionary[onehot_pred_index])
            symbols_in_keys = symbols_in_keys[1:]
            symbols_in_keys.append(onehot_pred_index)
        print ('-'*50)
        print ('TEXT GENERATED')
        print(sentence)

INFO:tensorflow:Restoring parameters from ./esp/checkpoints/model_final
3 words: vayan vayan vayan
sentence:  vayan vayan vayan
['vayan', 'vayan', 'vayan']
--------------------------------------------------
TEXT GENERATED
vayan vayan vayan se es chile lujo debería de desgarrador se esta está en es de padres no esta está en es de padres no esta está en es de padres no esta está en


KeyboardInterrupt: 